In [8]:
import nltk
nltk.download('reuters')
from nltk.corpus import reuters
import gensim
import logging

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [9]:
reuters.categories()

['acq',
 'alum',
 'barley',
 'bop',
 'carcass',
 'castor-oil',
 'cocoa',
 'coconut',
 'coconut-oil',
 'coffee',
 'copper',
 'copra-cake',
 'corn',
 'cotton',
 'cotton-oil',
 'cpi',
 'cpu',
 'crude',
 'dfl',
 'dlr',
 'dmk',
 'earn',
 'fuel',
 'gas',
 'gnp',
 'gold',
 'grain',
 'groundnut',
 'groundnut-oil',
 'heat',
 'hog',
 'housing',
 'income',
 'instal-debt',
 'interest',
 'ipi',
 'iron-steel',
 'jet',
 'jobs',
 'l-cattle',
 'lead',
 'lei',
 'lin-oil',
 'livestock',
 'lumber',
 'meal-feed',
 'money-fx',
 'money-supply',
 'naphtha',
 'nat-gas',
 'nickel',
 'nkr',
 'nzdlr',
 'oat',
 'oilseed',
 'orange',
 'palladium',
 'palm-oil',
 'palmkernel',
 'pet-chem',
 'platinum',
 'potato',
 'propane',
 'rand',
 'rape-oil',
 'rapeseed',
 'reserves',
 'retail',
 'rice',
 'rubber',
 'rye',
 'ship',
 'silver',
 'sorghum',
 'soy-meal',
 'soy-oil',
 'soybean',
 'strategic-metal',
 'sugar',
 'sun-meal',
 'sun-oil',
 'sunseed',
 'tea',
 'tin',
 'trade',
 'veg-oil',
 'wheat',
 'wpi',
 'yen',
 'zinc']

In [0]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import gensim.models.word2vec as word2vec
import copy
from tqdm import tnrange, tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
print(f"Num tags: {len(reuters.categories())}")
print(f"NUm Docs: {len(reuters.fileids())}")

Num tags: 90
NUm Docs: 10788


# Loading the data

In [33]:
nltk.download('stopwords')
stop_words = stopwords.words("english") #common words wthout much meaning

documents = reuters.fileids()

train_set = [doc for doc in documents if doc.startswith('training')] #group of train docs
test_set = [doc for doc in documents if doc.startswith('test')]

dataset = {}

dataset['train'] = [reuters.raw(doc_id) for doc_id in train_set]
dataset['test'] = [reuters.raw(doc_id) for doc_id in test_set]

print(f"Train sentences: {len(dataset['train'])}")
print(f"Test sentences: {len(dataset['test'])}")

# merging train and test for preprocessing
total_dataset = copy.deepcopy(dataset['train'])
total_dataset.extend(dataset['test'])

# converting all text to lowercase

for i in tnrange(len(total_dataset), desc='Lower Case'):
  total_dataset[i] = str(total_dataset[i]).lower()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Train sentences: 7769
Test sentences: 3019


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`


## Convert sentences to words

In [0]:
import re

def sentence_to_wordlist(sentence):
  
  # breaks sentences into tokens using a regular expression pattern

  regex_pattern = re.sub("[^a-zA-Z0-9]", " ", sentence)
  words = regex_pattern.split()
  return words

In [38]:
# Test for sentence to wordlist

sentence_to_wordlist("Hey, this is Ayush Jain!")

['Hey', 'this', 'is', 'Ayush', 'Jain']

In [0]:
words = []

for sentence in total_dataset:
  words.append([w for w in sentence_to_wordlist(sentence) if w not in stop_words])

# Vocabulary builder

In [48]:
vocab_dup = [] #vocabulary with duplicates

for word in words:
  vocab_dup.extend(word)

vocab = list(set(vocab_dup)) #final vocabulary

print(f"Total number of tokens: {len(vocab_dup)}")
print(f"Vocabulary size (unique tokens) : {len(vocab)}")

print(f"Voacb percent total token size: {len(vocab)/len(vocab_dup) * 100}%")

Total number of tokens: 1021233
Vocabulary size (unique tokens) : 30806
Voacb percent total token size: 3.0165496022944813%


In [49]:
# sentence size for consistency

max_size = max([len(word) for word in words])
min_size = min([len(word) for word in words])

print(f"Max size of sentence: {max_size}, Min size of sentence: {min_size}")

Max size of sentence: 2280, Min size of sentence: 6
